# <center><h1>MultiTaskElastic-Net</h1><center>

<p> <center>Para entendermos <code>MultiTaskElastic-Net</code> precisamos primeiro ver sobre <code>Elastic-Net</code></center>

<h1><center>Elastic-Net<center></h1>
<p><center><code>Elastic-Net</code> é uma combinação de dois dos mais populares regularizadores de variantes da regressão linear:</center> <center><code>Ridge</code> e <code>Lasso</code>.</center> 

<h1><center><code>Ridge</code> utiliza uma penalidade L2:<center></h1>


<img src="https://i.ibb.co/KmZtKsN/ridge.png"> 

<h1><center><code>Lasso</code> utiliza uma penalidade L1:<center></h1>

<img src="https://i.ibb.co/br7yKMQ/lasso.png">

 <h1><center>Com Elastic-Net nós não precisamos escolher entre esses dois modelos, pois ela usa ambos, L1 e L2, como penalidade.<center></h1>

<img src="https://i.ibb.co/1zdrKcR/elastic-net.png">    

<p><center>Onde: 0 <= l1_ratio <= 1. Para l1_ratio = 0 a penalidade é um L2. Para l1_ratio = 1 é uma penalidade L1. Para 0 < l1_ratio < 1, a penalidade será uma combinação de L1 e L2</center></p> 

<h1><center>Enquato na Multitask temos:<center></h1>

<img src="https://i.ibb.co/0Vwrkyg/multitask.jpg">

<p><center>Onde: 0 < l1_ratio <= 1. Para l1_ratio = 1 a penalidade é L1/L2. Para l1_ratio = 0 a penalidade é L2. Para 0 < l1_ratio < 1, a penalidade é a combinação de L1/L2 e L2.</center></p>   

<h1><center>Para comparmos de forma simples podemos visualizar abaixo:</center></h1>

<img src="https://i.ibb.co/gwQDXrj/comparativo.png">

<p><center>Entretanto, por utilizar ambos os modelos <code>Lasso</code> e <code>Ridge</code>, o <code>Elastic-Net</code> demanda um processamento muito grande, da mesma forma que o <code>MultiTaskElastic-Net</code></center></p>    

# Treinando o modelo proposto

## 01 - Importando Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse_
from sklearn.linear_model import MultiTaskElasticNet
import warnings
warnings.filterwarnings('ignore')

## 02 - Importando dados necessários

In [ ]:
df = pd.read_csv('data/data.csv', delimiter = '\t').rename(columns = {'<DATE>': 'date', '<OPEN>': 'open', '<HIGH>':'high', '<CLOSE>':'close', '<LOW>':'low'}).drop(columns = ['<TICKVOL>', '<VOL>', '<SPREAD>'])
df['adj_close'] = df['close'] * 0.8375918 # criando coluna de adf_close para futuros calculos

## 03 - Definindo RSI/IFR

In [ ]:
def rsi(data, column, window=1):
    """Retorn o calculo de RSI sobre o DF e coluna"""   
    
    data = data.copy()
    
    # Establish gains and losses for each day
    data["Variation"] = data[column].diff()
    data = data[1:]
    data["Gain"] = np.where(data["Variation"] > 0, data["Variation"], 0)
    data["Loss"] = np.where(data["Variation"] < 0, data["Variation"], 0)

    # Calculate simple averages so we can initialize the classic averages
    simple_avg_gain = data["Gain"].rolling(window).mean()
    simple_avg_loss = data["Loss"].abs().rolling(window).mean()
    classic_avg_gain = simple_avg_gain.copy()
    classic_avg_loss = simple_avg_loss.copy()

    for i in range(window, len(classic_avg_gain)):
        classic_avg_gain[i] = (classic_avg_gain[i] * (window - 1) + data["Gain"].iloc[i]) / window
        classic_avg_loss[i] = (classic_avg_loss[i] * (window - 1) + data["Loss"].abs().iloc[i]) / window
    
    # Calculate the RSI
    RS = classic_avg_gain / classic_avg_loss
    RSI = 100 - (100 / (1 + RS))
    return RSI

## 04 - Definindo tabela com RSI/IFR

In [ ]:
def rsi_table(df):
    """Usa um DF criando uma coluna com o sinal para o ML"""
    df["IFR2"] = rsi(df, column="adj_close")
    df["Target1"] = df["high"].shift(1)
    df["Target2"] = df["high"].shift(2)
    df["Target"] = df[["Target1", "Target2"]].max(axis=1)
    df.drop(columns=["Target1", "Target2"], inplace=True)
    
    # Define exact buy price
    rsi_parameter = 40
    df["buy_price"] = np.where(df["IFR2"] <= rsi_parameter, df["close"], 0)

    # Define exact sell price
    df["sell_price"] = np.where(
        df["high"] > df['Target'], 
        np.where(df['open'] > df['Target'], df['open'], df['Target']),
        0) 
    trade_logic = []
    for i in range(len(df['close'])):
        if ((df['buy_price'][i] == 0) & (df['sell_price'][i] == 0)) == True:
            trade_logic.append(-1)
        elif (df['buy_price'][i] != 0) == True:
            trade_logic.append(1)
        else:
            trade_logic.append(0)
    df['sinal'] = trade_logic
    df = df.drop(columns = {'IFR2', 'Target', 'buy_price', 'sell_price'})
    return df

## 05 - Separando os dados (treino e teste)

In [ ]:
df = rsi_table(df)
y = df.sinal
X = df.drop(columns = ['sinal', 'date'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3369, random_state = 42)

## 06 - Treinando o modelo

In [ ]:
mt_elastic_net = MultiTaskElasticNet()
mt_elastic_net.fit(X_train, y_train[:, np.newaxis])
y_pred = mt_elastic_net.predict(X_test)
mse_(y_test, y_pred)
mt_elastic_net.score(X_test, y_test)

# Criando link com MetaTrader5
## 01 - Importando bibliotecas necessárias, definindo funcao para importar datasets

In [ ]:
import MetaTrader5 as mt
from datetime import datetime    
# Initializing MT5 connection 
def input_mt5(x_1, x_2, x_3):
    """x = moeda, y = TIMEFRAME, z = range"""
    mt.initialize()
    mt.terminal_info()

    print("Inicialização bem sucedida: {}".format(mt.initialize()))
    # Copying data to pandas data frame
    stockdata = pd.DataFrame()
    rates = mt.copy_rates_from_pos(x_1, x_2, 0, x_3)
    # Deinitializing MT5 connection
    mt.shutdown()
    print("Desligamento bem sucedido: {}".format(mt.shutdown()))
    stockdata['open'] = [y['open'] for y in rates]
    stockdata['high'] = [y['high'] for y in rates]
    stockdata['low'] = [y['low'] for y in rates]
    stockdata['close'] = [y['close'] for y in rates]
    stockdata['adj_close'] = stockdata.close * 0.8375918 
    
    return stockdata

## 02 - Definindo ordem de venda

In [ ]:
def buy(x, _vol):
    price = mt.symbol_info_tick(x).ask
    
    sl  = price - 80.0*mt.symbol_info(x).point
    tp  = price + 100.0*mt.symbol_info(x).point

    request = {
        "action": mt.TRADE_ACTION_DEAL,
        "symbol": x,
        "volume": _vol,
        "type": mt.ORDER_TYPE_BUY,
        "sl": sl,
        "tp": tp,
        "magic": 124512,
        "deviation": 0,
        "comment": "Buy Order",
        "type_time": mt.ORDER_TIME_GTC,
        "type_filling": mt.ORDER_FILLING_FOK,
    }

    result = mt.order_send(request)
    print(x)
    print(f'OrderSended buy: {result}')

## 03 - Definindo a função de compra

In [ ]:
def sell(x, _vol):
    price = mt.symbol_info_tick(x).bid

    sl  = price + 80.0*mt.symbol_info(x).point
    tp  = price - 100.0*mt.symbol_info(x).point

    request = {
        "action": mt.TRADE_ACTION_DEAL,
        "symbol": x,
        "volume": _vol,
        "type": mt.ORDER_TYPE_SELL,
        "sl": sl,
        "tp": tp,
        "magic": 124512,
        "deviation": 0,
        "comment": "Sell Order",
        "type_time": mt.ORDER_TIME_GTC,
        "type_filling": mt.ORDER_FILLING_FOK,
    }

    result = mt.order_send(request)

    print(f'OrderSended sell: {result}')

# 04 - Realização da compra com o ML em cima dos dados importados usando a funcao input_mt5

In [ ]:
mt.initialize()
last_price = input_mt5('WINQ21', mt.TIMEFRAME_M5, 5)
mt.initialize()

while True:
    predict_result = mt_elastic_net.predict(last_price)
    if mt.positions_get(symbol="WINQ21") == ():
        if predict_result[-1] >= .14 :
            buy("WINQ21", 1.0)
            mt.shutdown()
            break

        else:
            sell("WINQ21", 1.0)
            print('Nao pode dar predict')
            mt.shutdown()
            break
    else:
        print('Erro')
        break